In [2]:
#This project is about music composition using AI
#We mainly focused on the music of Piano
#We used LSTM, a Recurrent Neural Network(RNN) approach
#Platform : Google Colab
#Libraries : Tensorflow,Music21,Keras,NumPy,Sklearn,tqdm

In [1]:
from google.colab import files
#upload zip file of All_Midi_Files given 
path_to_file = list(files.upload().keys())[0]

Saving All_Midi_Files.zip to All_Midi_Files.zip


In [3]:
!apt-get install poppler-utils
!unzip /content/All_Midi_Files.zip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 154 kB of archives.
After this operation, 613 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 poppler-utils amd64 0.62.0-2ubuntu2.12 [154 kB]
Fetched 154 kB in 1s (125 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 155653 files and directories currently installed.)
Preparing to unpack .../poppler-utils_0.62.0-2ubuntu2.12_amd64.deb ...
Unpacking poppler-utils (0.62.0-2ubuntu2.12) ...
Setting up poppler-utils (0.62.0-2ubuntu2.12) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Archive:  /content/All_Midi_Files.zip
 extract

In [4]:
from music21 import *
import glob
import numpy as np
import pandas as pd
from tqdm import tqdm
from tensorflow.keras.layers import LSTM,Dense,Input,Dropout
from tensorflow.keras.models import Sequential,Model,load_model
from sklearn.model_selection import train_test_split
import random

In [5]:
#Reading and parsing function
def read_file(file):
  notes=[]
  notes_to_parse=None
  midi=converter.parse(file)
  instrmt=instrument.partitionByInstrument(midi)

  #Fetching Piano Data
  for part in instrmt.parts:
    if 'Piano' in str(part):
      notes_to_parse=part.recurse()

#checking element type is Note or chord
 # if element is chord, we split it into notes
      for element in notes_to_parse:
        if type(element)==note.Note:
          notes.append(str(element.pitch))
        elif type(element)==chord.Chord:
          notes.append('.'.join(str(n) for n in element.normalOrder))
  return notes

In [6]:
file_path=["haydn"]
all_files=glob.glob('All Midi Files/'+file_path[0]+'/*.mid', recursive=True)

#reading each midi file
notes_array= np.array([read_file(i) for i in tqdm(all_files, position=0,leave=True)])


100%|██████████| 21/21 [01:18<00:00,  3.71s/it]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """


In [7]:
#making array of unique notes
notess = sum(notes_array,[])
unique_notes = list(set(notess))
print("Unique Notes:",len(unique_notes))

#notes with their frequency
freq = dict(map(lambda x: (x,notess.count(x)),unique_notes))

#getting the threshold frequency
print("\nFrequency notes")
for i in range(30,100,20):
  print(i,":",len(list(filter(lambda x:x[1]>=i,freq.items()))))
#freq_notes = []

#filtering notes >50
freq_notes = dict(filter(lambda x:x[1]>=50, freq.items()))
new_notes = [[i for i in j if i in freq_notes] for j in notes_array]

#dictionary having key as note index and value as note
ind2note = dict(enumerate(freq_notes))

#reverse of above dictionary
note2ind = dict(map(reversed,ind2note.items()))



Unique Notes: 155

Frequency notes
30 : 76
50 : 64
70 : 56
90 : 48


In [8]:
timesteps=50

#store values of input and output
x=[] ; y=[]

for i in new_notes:
  for j in range(0,len(i)-timesteps):
    #input will be the current index + timestep
    #output will be the next index after timestep
    inp=i[j:j+timesteps] ; out=i[j+timesteps]

    #append the index value of respective notes 
    x.append(list(map(lambda x:note2ind[x],inp)))
    y.append(note2ind[out])

x_new=np.array(x) 
y_new=np.array(y)
print(x_new.shape)

(25392, 50)


In [9]:
x_new = np.reshape(x_new,(len(x_new),timesteps,1))
y_new = np.reshape(y_new,(-1,1))

#splitting the input values into training and testing sets in 80:20 ratio
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x_new,y_new,test_size=0.2, random_state =42 )

In [10]:
#creating the model
model=Sequential()

#creating 2 stacked LSTM layer with dimension 256
model.add(LSTM(256,return_sequences=True,input_shape=(x_new.shape[1],x_new.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(256,activation='relu'))

#fully connected layer for the output with softmax activation
model.add(Dense(len(note2ind),activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50, 256)           264192    
                                                                 
 dropout (Dropout)           (None, 50, 256)           0         
                                                                 
 lstm_1 (LSTM)               (None, 256)               525312    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense (Dense)               (None, 256)               65792     
                                                                 
 dense_1 (Dense)             (None, 64)                16448     
                                                                 
Total params: 871,744
Trainable params: 871,744
Non-trai

In [11]:
#compiling model with adam optimizer
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

#training model on training,validation and testing sets
model.fit(x_train,y_train,batch_size=128,epochs=80,validation_data=(x_test,y_test))

#saving model
model.save("MOD")



Epoch 1/80
159/159 [==============================] - 11s 23ms/step - loss: 3.7272 - accuracy: 0.0581 - val_loss: 3.6643 - val_accuracy: 0.0760
Epoch 2/80
159/159 [==============================] - 3s 18ms/step - loss: 3.6202 - accuracy: 0.0774 - val_loss: 3.5849 - val_accuracy: 0.0918
Epoch 3/80
159/159 [==============================] - 3s 18ms/step - loss: 3.5230 - accuracy: 0.0913 - val_loss: 3.4619 - val_accuracy: 0.1207
Epoch 4/80
159/159 [==============================] - 3s 18ms/step - loss: 3.4092 - accuracy: 0.1120 - val_loss: 3.3759 - val_accuracy: 0.1347
Epoch 5/80
159/159 [==============================] - 3s 18ms/step - loss: 3.2950 - accuracy: 0.1370 - val_loss: 3.2435 - val_accuracy: 0.1492
Epoch 6/80
159/159 [==============================] - 3s 18ms/step - loss: 3.1484 - accuracy: 0.1646 - val_loss: 3.1291 - val_accuracy: 0.1841
Epoch 7/80
159/159 [==============================] - 3s 19ms/step - loss: 2.9904 - accuracy: 0.1977 - val_loss: 2.9859 - val_accuracy: 0.210

INFO:tensorflow:Assets written to: MOD/assets


INFO:tensorflow:Assets written to: MOD/assets


In [12]:
#loading model from saved models
model=load_model("MOD")

#generating random index
index = np.random.randint(0,len(x_test)-1)

music_pattern=x_test[index]

# making empty list for predicted notes
out_pred=[]

#iterating till 200 notes is generated
for i in range(200): 
  #reshaping the music pattern
  music_pattern=music_pattern.reshape(1,len(music_pattern),1)

  #getting the note which has maximum probability of occurance
  pred_index = np.argmax(model.predict(music_pattern))
  out_pred.append(ind2note[pred_index])
  music_pattern = np.append(music_pattern,pred_index)

  #updating the music pattern with one timestamp ahead
  music_pattern = music_pattern[1:]

In [13]:
#saving the predicted notes in output_notes
output_notes = []
for offset,pattern in enumerate(out_pred):
  #if pattern is a chord instance
  if ('.' in pattern) or pattern.isdigit():
    #split notes from the chord
    notes_in_chord = pattern.split('.')
    notes = []
    for current_note in notes_in_chord:
        i_curr_note=int(current_note)
        #cast the current note to Note object and
        #append the current note 
        new_note = note.Note(i_curr_note)
        new_note.storedInstrument = instrument.Piano()
        notes.append(new_note)
    
    #cast the current note to Chord object
    #offset will be 1 step ahead from the previous note
    #as it will prevent notes to stack up 
    new_chord = chord.Chord(notes)
    new_chord.offset = offset
    output_notes.append(new_chord)
  
  else:
    #cast the pattern to Note object apply the offset and 
    #append the note
    new_note = note.Note(pattern)
    new_note.offset = offset
    new_note.storedInstrument = instrument.Piano()
    output_notes.append(new_note)

#save the midi file 
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='AI_composed_music.mid')



'AI_composed_music.mid'